# Прием и обработка твитов микробатчем

## Инициализация

In [1]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.toree.kernel.api
import java.util.Calendar

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [3]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_6cb85880407d


pipeline_6cb85880407d

## Определяем схему

In [4]:
val inputStreamPath = "/home/jovyan/work/events-stream"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("timestamp", TimestampType)

inputStreamPath = /home/jovyan/work/events-stream
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))


StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))

## Определяем стриминговый датасет и применяем к нему модель для предсказания

In [5]:
// Определяем udf для предсказания негативного (0) или позитивного (1) твита
val getPrediction01 =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector) =>
        if ( prediction(0) >= prediction(1) ) 0 else 1 // При равенстве будем склоняться к 0 :)
    )

val inputDF =
    model.transform( // применяем модель для предсказания
        spark
        .readStream
        .schema(dataSchema)
        .json(inputStreamPath)
        .withWatermark("timestamp", "60 seconds") // задаем время устаревания данных в потоке пока что, на всякий случай, гораздо больше, чем предположительно нужно
    )
    .withColumn("prediction01",getPrediction01($"probability")) // добавляем столбец с udf
    .selectExpr(
        "window(timestamp, '10 seconds', '5 seconds') as window", // определяем скользящее окно размером в 10 секунд со сдвигом в 5 секунд
        "case when prediction01 = 0 then 1 else 0 end as pred_neg", // это основной столбец, требуемый в домашнем задании (предсказание к-ва негативных твитов в окне)
        "case when prediction01 = 1 then 1 else 0 end as pred_pos", // дополнительный столбец для полноты картины :)
        // В этом кейсе мы можем себе позволить вывести также и реальные метки негатива и позитива для сравнения с предсказанными :)
        "case when hiddentargetclue = 0 then 1 else 0 end as clue_neg",
        "case when hiddentargetclue = 1 then 1 else 0 end as clue_pos"
    )
    .groupBy($"window") // собираем к-во твитов в каждом окне
    .agg( // и схлопываем "шахматный" отчет, суммируя соотв. столбцы
        sum("pred_neg").alias("predicted_neg"),
        sum("pred_pos").alias("predicted_pos"),
        sum("clue_neg").alias("clue_neg"),
        sum("clue_pos").alias("clue_pos")
    )

getPrediction01 = UserDefinedFunction(<function1>,IntegerType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
inputDF = [window: struct<start: timestamp, end: timestamp>, predicted_neg: bigint ... 3 more fields]


[window: struct<start: timestamp, end: timestamp>, predicted_neg: bigint ... 3 more fields]

## Определяем потоковый запрос и стартуем его, обрабатывая батчами

In [6]:
var fRuns = 0
// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита и доп. стоблцы
// В задании написано, что вероятность негатива - это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.outputMode("complete").foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            println(s"Batch run # $fRuns")
            batchDF.orderBy(($"window").desc).show(false)
        } catch {
            case e:Throwable => {
                println(e.getMessage)
            }
        }
    }
}.start()

Batch run # 1


fRuns = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4645ca42


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4645ca42

+------------------------------------------+-------------+-------------+--------+--------+
|window                                    |predicted_neg|predicted_pos|clue_neg|clue_pos|
+------------------------------------------+-------------+-------------+--------+--------+
|[2020-01-29 17:32:00, 2020-01-29 17:32:10]|11           |25           |12      |24      |
|[2020-01-29 17:31:55, 2020-01-29 17:32:05]|15           |35           |16      |34      |
|[2020-01-29 17:31:50, 2020-01-29 17:32:00]|8            |19           |7       |20      |
|[2020-01-29 17:31:45, 2020-01-29 17:31:55]|10           |26           |11      |25      |
|[2020-01-29 17:31:40, 2020-01-29 17:31:50]|9            |25           |13      |21      |
|[2020-01-29 17:31:35, 2020-01-29 17:31:45]|5            |13           |7       |11      |
|[2020-01-29 17:31:30, 2020-01-29 17:31:40]|2            |5            |2       |5       |
|[2020-01-29 17:31:25, 2020-01-29 17:31:35]|5            |8            |7       |6       |

## Останов чтения потока

In [7]:
stream.stop()